In [ ]:
import pandas as pd
import glob
import os


ML = True #False : DL 
SUB_AGG = False #FALSE : sub-wise 

def get_path_to_results(ML, SUB_AGG):
    path_to_results = './results/'
    path_to_results += 'ML/' if ML else 'DL/'
    path_to_results += 'sub_agg/' if SUB_AGG else 'sub_wise/'
    return path_to_results


#separate into aggregate and take_stat
def aggregate_and_take_stat(result_csv_root_path,
                            result_file_name_prefix="results_seed-",
                            seed_ranges=(0,49),
                            metrics=['accuracy', 'roc_auc', 'f1_score', 'balanced_accuracy'],
                            sub_agg = False):
    if sub_agg : 
        #! TODO ;IMPLMENET THE SUBWISE AGGREGATION (FOR LOOP OVER SUBS OR STH AND COMBINE THE COMBINED_DF OR STH!)
        #use aggregate_results and take_stats functions here
        combined_df = aggregate_seed_results(result_csv_root_path,
                            result_file_name_prefix=result_file_name_prefix,
                            seed_ranges=seed_ranges)
        
    else:
        combined_df_across_subs = []
        import pdb ; pdb.set_trace()
        sub_dirs = glob.glob(result_csv_root_path + '/subject-*/')
        for sub_dir in sub_dirs:
            sub_path = sub_dir
            combined_df_sub = aggregate_seed_results(sub_path,
                            result_file_name_prefix=result_file_name_prefix,
                            seed_ranges=seed_ranges)
            combined_df_across_subs.append(combined_df_sub)
        combined_df = pd.concat(combined_df_across_subs, ignore_index=True)
        
            
    assert combined_df is not None, "No results found"
    result_df = take_stats(combined_df, metrics=metrics)
    return result_df


def take_stats(df, group_by_cols = ['model_name', 'data_name'],
               metrics=['accuracy', 'roc_auc', 'f1_score', 'balanced_accuracy']):
    """
    Calculate mean, std, min, max for each metric, after grouping by group_by_cols
    """
    # Group and calculate statistics
    grouped = df.groupby(group_by_cols)
    
    # Calculate means and stds
    stats_dict = {}
    for metric in metrics:
        stats_dict[f'{metric}_mean'] = grouped[metric].mean()
        stats_dict[f'{metric}_std'] = grouped[metric].std()
        stats_dict[f'{metric}_min'] = grouped[metric].min()
        stats_dict[f'{metric}_max'] = grouped[metric].max()
    
    # Create final DataFrame
    result_df = pd.DataFrame(stats_dict).reset_index()
    
    return result_df.sort_values(group_by_cols)
    
    
def aggregate_seed_results(result_csv_root_path,
                        result_file_name_prefix="results_seed-",
                        seed_ranges=(0,49)):    
    # Read and combine all files
    df_list = []
    for seed in range(seed_ranges[0], seed_ranges[1]+1):
        file_path = os.path.join(result_csv_root_path,
                                f"{result_file_name_prefix}{seed}.csv")
        if os.path.exists(file_path):
            df_list.append(pd.read_csv(file_path))
    
    assert len(df_list) == seed_ranges[1] - seed_ranges[0] + 1, f"Some files are missing, expected {seed_ranges[1] - seed_ranges[0] + 1} csv files, got {len(df_list)}"
        
    combined_df = pd.concat(df_list, ignore_index=True)
    return combined_df      
    

#test the function
path_to_results = get_path_to_results(ML, SUB_AGG)
print(path_to_results)
aa = aggregate_and_take_stat(path_to_results, seed_ranges=(0, 49), sub_agg = SUB_AGG)


./results/ML/sub_wise/
> /tmp/ipykernel_708256/2950728468.py(32)aggregate_and_take_stat()
     30         combined_df_across_subs = []
     31         import pdb ; pdb.set_trace()
---> 32         sub_dirs = glob.glob(result_csv_root_path + '/subject-*/')
     33         for sub_dir in sub_dirs:
     34             sub_path = sub_dir

> /tmp/ipykernel_708256/2950728468.py(33)aggregate_and_take_stat()
     31         import pdb ; pdb.set_trace()
     32         sub_dirs = glob.glob(result_csv_root_path + '/subject-*/')
---> 33         for sub_dir in sub_dirs:
     34             sub_path = sub_dir
     35             combined_df_sub = aggregate_seed_results(sub_path,

> /tmp/ipykernel_708256/2950728468.py(34)aggregate_and_take_stat()
     32         sub_dirs = glob.glob(result_csv_root_path + '/subject-*/')
     33         for sub_dir in sub_dirs:
---> 34             sub_path = sub_dir
     35             combined_df_sub = aggregate_seed_results(sub_path,
     36                         

AssertionError: Some files are missing, expected 50 csv files, got 49

In [5]:
aa #'./results/ML/sub_agg/'


,model_name,data_name,accuracy_mean,accuracy_std,accuracy_min,accuracy_max,roc_auc_mean,roc_auc_std,roc_auc_min,roc_auc_max,f1_score_mean,f1_score_std,f1_score_min,f1_score_max,balanced_accuracy_mean,balanced_accuracy_std,balanced_accuracy_min,balanced_accuracy_max
0,BayesianGaussianMixture,PCA_10-base,0.521667,0.136924,0.305556,0.833333,0.507491,0.053736,0.367273,0.730769,0.308128,0.232282,0.00,0.625000,0.507491,0.053736,0.367273,0.730769
1,BayesianGaussianMixture,PCA_10-fixation_denormed,0.555556,0.132447,0.277778,0.777778,0.522973,0.049152,0.444805,0.655518,0.287976,0.241308,0.00,0.600000,0.522973,0.049152,0.444805,0.655518
2,BayesianGaussianMixture,PCA_10-fixation_normalized,0.461111,0.137693,0.277778,0.777778,0.475352,0.059832,0.279221,0.555556,0.334254,0.217224,0.00,0.690909,0.475352,0.059832,0.279221,0.555556
3,BayesianGaussianMixture,PCA_10-halfnfft,0.503333,0.132879,0.250000,0.777778,0.504184,0.053364,0.389610,0.678571,0.360128,0.217701,0.00,0.648649,0.504184,0.053364,0.389610,0.678571
4,BayesianGaussianMixture,PCA_10-halfoverlap,0.528889,0.129679,0.305556,0.777778,0.513366,0.065794,0.391304,0.690476,0.322779,0.259673,0.00,0.647059,0.513366,0.065794,0.391304,0.690476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,SVC,nooverlap,0.481111,0.069967,0.277778,0.611111,0.441640,0.074551,0.250836,0.592593,0.277164,0.100189,0.08,0.484848,0.441640,0.074551,0.250836,0.592593
1041,SVC,nooverlap_halfnfft,0.481111,0.069967,0.277778,0.611111,0.441640,0.074551,0.250836,0.592593,0.277164,0.100189,0.08,0.484848,0.441640,0.074551,0.250836,0.592593
1042,SVC,raw,0.481111,0.069967,0.277778,0.611111,0.441640,0.074551,0.250836,0.592593,0.277164,0.100189,0.08,0.484848,0.441640,0.074551,0.250836,0.592593
1043,SVC,whole_trial_denormed,0.481111,0.069967,0.277778,0.611111,0.441640,0.074551,0.250836,0.592593,0.277164,0.100189,0.08,0.484848,0.441640,0.074551,0.250836,0.592593


In [ ]:
import pandas as pd
import glob
import os


ML = True #False : DL 
SUB_AGG = True #FALSE : sub-wise 




def get_path_to_results(ML, SUB_AGG):
    path_to_results = './results/'
    path_to_results += 'ML/' if ML else 'DL/'
    path_to_results += 'sub_agg/' if SUB_AGG else 'sub_wise/'
    return path_to_results


#separate into aggregate and take_stat
def aggregate_seed_results(result_csv_root_path, 
                         result_file_name_prefix="results_seed-",
                         seed_ranges=(0,49)):
    """
    Aggregate results across seeds into mean/std statistics
    """

    # Get all CSV files for seeds
    pattern = os.path.join(result_csv_root_path, 
                          f"{result_file_name_prefix}*.csv")
    all_files = glob.glob(pattern)
    
    # Read and combine all files
    df_list = []
    for seed in range(seed_ranges[0], seed_ranges[1]+1):
        file_path = os.path.join(result_csv_root_path,
                                f"{result_file_name_prefix}{seed}.csv")
        if os.path.exists(file_path):
            df_list.append(pd.read_csv(file_path))
    
    assert len(df_list) == seed_ranges[1] - seed_ranges[0] + 1, f"Some files are missing, expected {seed_ranges[1] - seed_ranges[0] + 1} csv files, got {len(df_list)}"
        
    combined_df = pd.concat(df_list, ignore_index=True)
    
    import pdb; pdb.set_trace()
    
    # Define metrics to aggregate
    metrics = ['accuracy', 'roc_auc', 'f1_score', 'balanced_accuracy']
    
    # Group and calculate statistics
    grouped = combined_df.groupby(['model_name', 'data_name'])
    
    
    # Calculate means and stds
    stats_dict = {}
    for metric in metrics:
        stats_dict[f'{metric}_mean'] = grouped[metric].mean()
        stats_dict[f'{metric}_std'] = grouped[metric].std()
        stats_dict[f'{metric}_min'] = grouped[metric].min()        
    
    # Create final DataFrame
    result_df = pd.DataFrame(stats_dict).reset_index()
    
    return result_df.sort_values(['model_name', 'data_name'])

def subwise_aggregate_seed_results(result_csv_root_path,
                                   result_file_name_prefix="results_seed-",
                                   seed_ranges=(0,49),
                                   subwise_ranges=(0,2)):
    #use the aggregate_seed_results function, but on a subwise basis and average across the subejct too
    subwise_aggregate = []
    for sub in range(subwise_ranges[0], subwise_ranges[1]+1):
        path_to_results = result_csv_root_path + f'subject-{sub}/'
        subject_i_df = aggregate_seed_results(path_to_results, 
                                    result_file_name_prefix=result_file_name_prefix,
                                    seed_ranges=(0, 49))
        subwise_aggregate.append(subject_i_df)
    subwise_aggregate = pd.concat(subwise_aggregate, ignore_index=True)
    grouped = subwise_aggregate.groupby(['model_name', 'data_name'])
        
        
                
    
    

#test the function
path_to_results = get_path_to_results(ML, SUB_AGG)
aa = aggregate_seed_results(path_to_results, seed_ranges=(0, 49))
